In [54]:
import pandas as pd
import os
import xarray as xr

In [55]:
# Ordnerpfad zu den CSV-Dateien
csv_folder = 'data/Json_Dateien/'

# Leere Liste zum Speichern der DataFrames
dataframes = []

# Alle CSV-Dateien im Ordner einlesen
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        csv_file = os.path.join(csv_folder, filename)  # Vollständiger Pfad zur CSV-Datei
        df = pd.read_csv(csv_file)  # Einlesen der CSV-Datei
        dataframes.append(df)  # Hinzufügen des DataFrames zur Liste

# Alle DataFrames zu einem einzigen DataFrame zusammenführen
df = pd.concat(dataframes, ignore_index=True)

In [56]:
df.head(2)

,id,dataset,mrid,revisionNumber,publishTime,createdTime,messageType,messageHeading,eventType,unavailabilityType,...,fuelType,normalCapacity,availableCapacity,unavailableCapacity,eventStatus,eventStartTime,eventEndTime,cause,relatedInformation,outageProfile
0,60924,REMIT,11XDONG-PT-----2-NGET-RMT-00001020,2,2020-11-26T13:54:00Z,2020-11-26T13:54:00Z,UnavailabilitiesOfElectricityFacilities,REMIT Information,Production unavailability,Planned,...,Wind Offshore,400,0,400,Active,2020-11-26T09:00:00Z,2020-11-26T18:00:00Z,Planned Outage,HOW01 Z12 Dry run interlink test,"[{'startTime': '2020-11-26T09:00:00Z', 'endTim..."
1,185054,REMIT,11XDONG-PT-----2-NGET-RMT-00001024,2,2020-12-10T16:12:00Z,2020-12-10T16:12:00Z,UnavailabilitiesOfElectricityFacilities,REMIT Information,Production unavailability,Planned,...,Wind Offshore,400,0,400,Active,2020-12-11T06:00:00Z,2020-12-11T16:00:00Z,Planned Outage,HOW01 BMU 2 will be undergoing frequency respo...,"[{'startTime': '2020-12-11T06:00:00Z', 'endTim..."


In [57]:
# Konvertiere Zeitstempel in Datetime-Objekte
df['publishTime'] = pd.to_datetime(df['publishTime'])  
df['createdTime'] = pd.to_datetime(df['createdTime'])  
df['eventStatus'] = pd.to_datetime(df['eventStatus'], errors='coerce')  
df['eventStartTime'] = pd.to_datetime(df['eventStartTime'], errors='coerce')

C:\Users\Michael Jäckle\AppData\Local\Temp\ipykernel_24152\2432464020.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['eventStatus'] = pd.to_datetime(df['eventStatus'], errors='coerce')


In [58]:
df.dtypes

id                                   int64
dataset                             object
mrid                                object
revisionNumber                       int64
publishTime            datetime64[ns, UTC]
createdTime            datetime64[ns, UTC]
messageType                         object
messageHeading                      object
eventType                           object
unavailabilityType                  object
participantId                       object
registrationCode                    object
assetId                             object
assetType                           object
affectedUnit                        object
affectedUnitEIC                     object
affectedArea                        object
biddingZone                         object
fuelType                            object
normalCapacity                       int64
availableCapacity                    int64
unavailableCapacity                  int64
eventStatus                 datetime64[ns]
eventStartT

In [59]:
# Filtere relevante Spalten aus dem DataFrame
df_filter = df.filter(['publishTime','normalCapacity','availableCapacity','unavailableCapacity'])
df_filter.head()

,publishTime,normalCapacity,availableCapacity,unavailableCapacity
0,2020-11-26 13:54:00+00:00,400,0,400
1,2020-12-10 16:12:00+00:00,400,0,400
2,2020-12-13 19:13:00+00:00,400,0,400
3,2020-12-07 21:19:00+00:00,400,0,400
4,2020-12-14 09:29:00+00:00,400,0,400


In [60]:
# Entferne doppelte Zeilen aus dem gefilterten DataFrame
df_filter_dup=df_filter.drop_duplicates()
df_filter_dup.head(2)

,publishTime,normalCapacity,availableCapacity,unavailableCapacity
0,2020-11-26 13:54:00+00:00,400,0,400
1,2020-12-10 16:12:00+00:00,400,0,400


In [72]:
# Runden der 'publishTime'-Spalte nach unten und oben zur nächsten Stunde
df_filter_dup.loc[:, 'publishTime_ru_unten'] = df_filter_dup['publishTime'].dt.floor('h')  # Runden nach unten (zur Stunde)
df_filter_dup.loc[:, 'publishTime_ru_oben'] = df_filter_dup['publishTime'].dt.ceil('h')    # Runden nach oben (zur Stunde)

# Berechnung der Zeitdifferenz zwischen 'publishTime_auf' und 'publishTime'
df_filter_dup.loc[:, 'zeitdifferenz'] = df_filter_dup['publishTime_ru_oben'] - df_filter_dup['publishTime']


In [62]:
def pre_csv(df):
    df.dtm = pd.to_datetime(df.dtm)
    return df

def pre_dwd(df):
    df = df.to_dataframe().reset_index().rename(columns={"ref_datetime":"reference_time", "valid_datetime":"valid_time"})
    df.reference_time = df.reference_time.dt.tz_localize("UTC")
    df.valid_time = df.reference_time + df.valid_time * pd.Timedelta(1, "h")
    return df

def pre_ncep(df):
    return df



df_hornsea_0 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_hornsea_1_20200920_20231027.nc"))
df_hornsea_1 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_hornsea_1_20231027_20240108.nc"))
df_hornsea_2 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_hornsea_1_20240108_20240129.nc"))
df_hornsea_3 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_hornsea_1_20240129_20240519.nc"))

df_hornsea = pd.concat([df_hornsea_0, df_hornsea_1, df_hornsea_2, df_hornsea_3]).sort_values(["reference_time", "valid_time"]).reset_index(drop=True)
del df_hornsea_0, df_hornsea_1, df_hornsea_2, df_hornsea_3

df_hornsea = df_hornsea.groupby(["reference_time", "valid_time"]).mean().reset_index().drop(columns=["longitude", "latitude"])

df_hornsea = df_hornsea.set_index("valid_time").groupby(["reference_time"]).resample("30min").interpolate("linear").drop(columns="reference_time").reset_index()
df_hornsea.head()


df_0 = pre_csv(pd.read_csv("data/Energy_Data_20200920_20240118.csv"))
df_1 = pre_csv(pd.read_csv("data/Energy_Data_20240119_20240519.csv"))

df = pd.concat([df_0, df_1]).sort_values(["dtm"]).reset_index(drop=True)
del df_0, df_1

df["Wind_MWh_credit"] = 0.5*df["Wind_MW"] - df["boa_MWh"] # Umrechnen in MWh und Abzug von BOA (BOA ist die Drosselung, die schon in MW miteingerechnet ist. Es wird aber mehr Strom produziert und vergütet, dashalb muss BOA wieder draufgerechnet werde, BOA ist immer negativ)
df["Solar_MWh_credit"] = 0.5*df["Solar_MW"]

df.head()

,dtm,MIP,Solar_MW,Solar_capacity_mwp,Solar_installedcapacity_mwp,Wind_MW,SS_Price,boa_MWh,DA_Price,Wind_MWh_credit,Solar_MWh_credit
0,2020-09-20 00:00:00+00:00,20.06,0.0,2130.537493,2228.208777,996.284,2.50,0.0,32.17,498.142,0.0
1,2020-09-20 00:30:00+00:00,19.77,0.0,2130.537493,2228.208777,957.576,15.00,0.0,32.17,478.788,0.0
2,2020-09-20 01:00:00+00:00,28.68,0.0,2130.537493,2228.208777,941.044,47.95,0.0,32.00,470.522,0.0
3,2020-09-20 01:30:00+00:00,28.97,0.0,2130.537493,2228.208777,964.366,29.13,0.0,32.00,482.183,0.0
4,2020-09-20 02:00:00+00:00,28.19,0.0,2130.537493,2228.208777,918.432,28.95,0.0,31.99,459.216,0.0


In [63]:
df_hornsea.dtypes

reference_time       datetime64[ns, UTC]
valid_time           datetime64[ns, UTC]
RelativeHumidity                 float32
Temperature                      float32
WindDirection                    float32
WindDirection:100                float32
WindSpeed                        float32
WindSpeed:100                    float32
dtype: object

In [64]:
df_hornsea['valid_time_round'] = df_hornsea['valid_time'].dt.floor('h').head()
df_hornsea['valid_time_up'] = df_hornsea['valid_time'].dt.floor('h')
df_hornsea.head(5)

,reference_time,valid_time,RelativeHumidity,Temperature,WindDirection,WindDirection:100,WindSpeed,WindSpeed:100,valid_time_round,valid_time_up
0,2020-09-20 00:00:00+00:00,2020-09-20 00:00:00+00:00,85.213745,15.416670,61.588081,62.085178,10.043627,11.802604,2020-09-20 00:00:00+00:00,2020-09-20 00:00:00+00:00
1,2020-09-20 00:00:00+00:00,2020-09-20 00:30:00+00:00,85.012253,15.412510,61.203667,61.726974,9.905537,11.648819,2020-09-20 00:00:00+00:00,2020-09-20 00:00:00+00:00
2,2020-09-20 00:00:00+00:00,2020-09-20 01:00:00+00:00,84.810768,15.408349,60.819256,61.368774,9.767447,11.495033,2020-09-20 01:00:00+00:00,2020-09-20 01:00:00+00:00
3,2020-09-20 00:00:00+00:00,2020-09-20 01:30:00+00:00,84.357880,15.451218,60.511028,61.111038,9.631039,11.354128,2020-09-20 01:00:00+00:00,2020-09-20 01:00:00+00:00
4,2020-09-20 00:00:00+00:00,2020-09-20 02:00:00+00:00,83.904999,15.494086,60.202801,60.853306,9.494630,11.213223,2020-09-20 02:00:00+00:00,2020-09-20 02:00:00+00:00


In [65]:
df_hornsea.head(3)

,reference_time,valid_time,RelativeHumidity,Temperature,WindDirection,WindDirection:100,WindSpeed,WindSpeed:100,valid_time_round,valid_time_up
0,2020-09-20 00:00:00+00:00,2020-09-20 00:00:00+00:00,85.213745,15.416670,61.588081,62.085178,10.043627,11.802604,2020-09-20 00:00:00+00:00,2020-09-20 00:00:00+00:00
1,2020-09-20 00:00:00+00:00,2020-09-20 00:30:00+00:00,85.012253,15.412510,61.203667,61.726974,9.905537,11.648819,2020-09-20 00:00:00+00:00,2020-09-20 00:00:00+00:00
2,2020-09-20 00:00:00+00:00,2020-09-20 01:00:00+00:00,84.810768,15.408349,60.819256,61.368774,9.767447,11.495033,2020-09-20 01:00:00+00:00,2020-09-20 01:00:00+00:00


In [68]:

df_merge = pd.merge( df_filter_dup, df_hornsea, right_on='valid_time_up',left_on='publishTime_ru_unten', how='right')

df_merge['faktor'] = df_merge['valid_time'].apply(lambda x: 30 if x.minute == 0 else 0)
df_merge['zeitdifferenz_float'] = df_merge['zeitdifferenz'].dt.total_seconds() / 60 
df_merge['zeit_pro_30_min']  = df_merge['zeitdifferenz_float'] - df_merge['faktor']
df_merge['zeit_pro_30_min'] = df_merge['zeit_pro_30_min'].apply(lambda x: 0 if x < 0 else (30 if x > 30 else x))
df_merge['anteil_in_prozent'] = 100/30*df_merge['zeit_pro_30_min']

df_merge.head()


,publishTime,normalCapacity,availableCapacity,unavailableCapacity,publishTime_up,publishTime_auf,zeitdifferenz,reference_time,valid_time,RelativeHumidity,...,WindDirection,WindDirection:100,WindSpeed,WindSpeed:100,valid_time_round,valid_time_up,faktor,zeitdifferenz_float,zeit_pro_30_min,anteil_in_prozent
0,NaT,NaN,NaN,NaN,NaT,NaT,NaT,2020-09-20 00:00:00+00:00,2020-09-20 00:00:00+00:00,85.213745,...,61.588081,62.085178,10.043627,11.802604,2020-09-20 00:00:00+00:00,2020-09-20 00:00:00+00:00,30,NaN,NaN,NaN
1,NaT,NaN,NaN,NaN,NaT,NaT,NaT,2020-09-20 00:00:00+00:00,2020-09-20 00:30:00+00:00,85.012253,...,61.203667,61.726974,9.905537,11.648819,2020-09-20 00:00:00+00:00,2020-09-20 00:00:00+00:00,0,NaN,NaN,NaN
2,NaT,NaN,NaN,NaN,NaT,NaT,NaT,2020-09-20 00:00:00+00:00,2020-09-20 01:00:00+00:00,84.810768,...,60.819256,61.368774,9.767447,11.495033,2020-09-20 01:00:00+00:00,2020-09-20 01:00:00+00:00,30,NaN,NaN,NaN
3,NaT,NaN,NaN,NaN,NaT,NaT,NaT,2020-09-20 00:00:00+00:00,2020-09-20 01:30:00+00:00,84.357880,...,60.511028,61.111038,9.631039,11.354128,2020-09-20 01:00:00+00:00,2020-09-20 01:00:00+00:00,0,NaN,NaN,NaN
4,NaT,NaN,NaN,NaN,NaT,NaT,NaT,2020-09-20 00:00:00+00:00,2020-09-20 02:00:00+00:00,83.904999,...,60.202801,60.853306,9.494630,11.213223,2020-09-20 02:00:00+00:00,2020-09-20 02:00:00+00:00,30,NaN,NaN,NaN


In [69]:
df_merge[df_merge['normalCapacity'].notna()]

,publishTime,normalCapacity,availableCapacity,unavailableCapacity,publishTime_up,publishTime_auf,zeitdifferenz,reference_time,valid_time,RelativeHumidity,...,WindDirection,WindDirection:100,WindSpeed,WindSpeed:100,valid_time_round,valid_time_up,faktor,zeitdifferenz_float,zeit_pro_30_min,anteil_in_prozent
70,2020-09-21 11:42:00+00:00,400.0,0.0,400.0,2020-09-21 11:00:00+00:00,2020-09-21 12:00:00+00:00,0 days 00:18:00,2020-09-20 00:00:00+00:00,2020-09-21 11:00:00+00:00,84.018257,...,206.259583,205.332825,2.223296,2.292838,NaT,2020-09-21 11:00:00+00:00,30,18.000000,0.000000,0.000000
71,2020-09-21 11:42:00+00:00,400.0,0.0,400.0,2020-09-21 11:00:00+00:00,2020-09-21 12:00:00+00:00,0 days 00:18:00,2020-09-20 00:00:00+00:00,2020-09-21 11:30:00+00:00,84.198746,...,208.600449,207.744919,2.490392,2.565651,NaT,2020-09-21 11:00:00+00:00,0,18.000000,18.000000,60.000000
158,2020-09-23 07:56:00+00:00,400.0,0.0,400.0,2020-09-23 07:00:00+00:00,2020-09-23 08:00:00+00:00,0 days 00:04:00,2020-09-20 00:00:00+00:00,2020-09-23 07:00:00+00:00,91.665131,...,203.355652,205.553635,7.900957,10.363153,NaT,2020-09-23 07:00:00+00:00,30,4.000000,0.000000,0.000000
159,2020-09-23 07:56:00+00:00,400.0,0.0,400.0,2020-09-23 07:00:00+00:00,2020-09-23 08:00:00+00:00,0 days 00:04:00,2020-09-20 00:00:00+00:00,2020-09-23 07:30:00+00:00,91.654205,...,201.029358,203.160522,7.874345,10.246035,NaT,2020-09-23 07:00:00+00:00,0,4.000000,4.000000,13.333333
168,2020-09-23 12:12:00+00:00,400.0,0.0,400.0,2020-09-23 12:00:00+00:00,2020-09-23 13:00:00+00:00,0 days 00:48:00,2020-09-20 00:00:00+00:00,2020-09-23 12:00:00+00:00,93.021278,...,183.616394,186.604691,9.251589,12.343545,NaT,2020-09-23 12:00:00+00:00,30,48.000000,18.000000,60.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1276411,2024-05-20 15:01:07+00:00,400.0,0.0,400.0,2024-05-20 15:00:00+00:00,2024-05-20 16:00:00+00:00,0 days 00:58:53,2024-05-19 00:00:00+00:00,2024-05-20 15:30:00+00:00,82.360435,...,7.700247,9.738106,6.276941,8.290260,NaT,2024-05-20 15:00:00+00:00,0,58.883333,30.000000,100.000000
1276444,2024-05-21 08:43:01+00:00,400.0,0.0,400.0,2024-05-21 08:00:00+00:00,2024-05-21 09:00:00+00:00,0 days 00:16:59,2024-05-19 00:00:00+00:00,2024-05-21 08:00:00+00:00,95.792404,...,62.673210,74.795410,3.833807,6.609612,NaT,2024-05-21 08:00:00+00:00,30,16.983333,0.000000,0.000000
1276445,2024-05-21 08:43:01+00:00,400.0,0.0,400.0,2024-05-21 08:00:00+00:00,2024-05-21 09:00:00+00:00,0 days 00:16:59,2024-05-19 00:00:00+00:00,2024-05-21 08:30:00+00:00,96.174713,...,67.017212,78.599045,3.823426,6.612945,NaT,2024-05-21 08:00:00+00:00,0,16.983333,16.983333,56.611111
1276542,2024-05-23 09:13:08+00:00,400.0,0.0,400.0,2024-05-23 09:00:00+00:00,2024-05-23 10:00:00+00:00,0 days 00:46:52,2024-05-19 00:00:00+00:00,2024-05-23 09:00:00+00:00,96.118858,...,57.504734,63.175957,8.391678,12.217297,NaT,2024-05-23 09:00:00+00:00,30,46.866667,16.866667,56.222222
